# Solution: Int GPU and the DevCloud

This notebook contains the solution to the exercise.



#### Set up paths so we can run Dev Cloud utilities
You *must* run this every time they enter a Workspace session.

In [1]:
%env PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support
import os
import sys
sys.path.insert(0, os.path.abspath('/opt/intel_devcloud_support'))
sys.path.insert(0, os.path.abspath('/opt/intel'))

env: PATH=/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/intel_devcloud_support


## The model

We will be using the `vehicle-license-plate-detection-barrier-0106` model for this exercise. Remember that to run a model on the GPU, we need to use FP16 as the model precision.

The model is present in the `/data/models/intel` folder.

# Step 1: Creating a Python Script

The first step is to create a python script that you can use to load the model and perform an inference. This is my version of the python script:

In [2]:
%%writefile inference.py

import time
import cv2
import numpy as np
from openvino.inference_engine import IENetwork
from openvino.inference_engine import IECore
import argparse

def main(args):
    model = args.model_path
    model_weights = model + '.bin'
    model_structure = model + '.xml'
    
    start = time.time()
    print("Starting...")
    print("Device:", args.device)
    print("Model:", args.model_path)

    core = IECore()
    print("Core created")
    
    network = IENetwork(model_structure, model_weights)
    print("Network created")

    exec_network = core.load_network(network, args.device, num_requests=1)
    print(f"Time taken to load model = {time.time()-start} seconds")

    # Reading and Preprocessing Image
    input_img=cv2.imread('car.png')
    input_img=cv2.resize(input_img, (300,300), interpolation = cv2.INTER_AREA)
    input_img=np.moveaxis(input_img, -1, 0)
    print("Image prepared")

    # Running Inference in a loop on the same image
    input_key = next(iter(network.inputs))
    input_dict = {input_key:input_img}
    print("Input created")

    start=time.time()
    for i in range(10):
        exec_network.infer(input_dict)
        print(f"#{i} Inference completed")
    
    print(f"Time Taken to run 10 Inferences on {args.device} is: {time.time()-start} seconds")

if __name__=='__main__':
    parser=argparse.ArgumentParser()
    parser.add_argument('--model_path', required=True)
    parser.add_argument('--device', default=None)
    
    args=parser.parse_args() 
    main(args)


Overwriting inference.py


## Step 2: Creating a job submission script

To submit a job to the devcloud, we need to create a script. I have named the script as `inference_cpu_model_job.sh` and in the cell below, you can find my version of the finished script:

In [3]:
%%writefile inference_job.sh

exec 1>/output/stdout.log 2>/output/stderr.log
mkdir -p /output

DEVICE=$1
MODELPATH=$2

# Run the load model python script
python3 inference.py  --model_path ${MODELPATH} --device ${DEVICE}

cd /output
tar zcvf output.tgz stdout.log stderr.log

Overwriting inference_job.sh


## Step 3: Running on the CPU or GPU

The cell below contains the command to submit jobs on the CPU or Int. GPU

In [4]:
DEVICE_CPU = 'CPU'
DEVICE_GPU = 'GPU'
MODEL_CPU = '/data/models/intel/vehicle-license-plate-detection-barrier-0106/FP32/vehicle-license-plate-detection-barrier-0106'
MODEL_GPU = '/data/models/intel/vehicle-license-plate-detection-barrier-0106/FP16/vehicle-license-plate-detection-barrier-0106'
NODE_CPU = 'tank-870:e3-1268l-v5'
NODE_GPU = 's002-n011:i5-6500te:intel-hd-530'

#job_id = !qsub inference_job.sh -d . -l nodes=1:{NODE_CPU} -F "{DEVICE_CPU} {MODEL_CPU}" -N store_core 
job_id = !qsub inference_job.sh -d . -l nodes=1:{NODE_GPU} -F "{DEVICE_GPU} {MODEL_GPU}" -N store_core
        
print(job_id[0])

J8zMylGtEmlZ8XaGTGEojSPpK4F4IPnq


## Step 5: Getting the Live Stat Values

By running the below command, we can see the live status of the commands.

In [ ]:
import liveQStat
liveQStat.liveQStat()

JobID,Status,Desc
0BfwOOVz0Jf2dnBKwu2Qd3ug1bCaKAgD,W,"Job is completed, but waiting on results."
0hNjDLRw98A0nWls9YUI5JNIOt4bLv2D,W,"Job is completed, but waiting on results."
1iVj41Xq7Hiv3LDPERYyRzHK01T3NDUO,W,"Job is completed, but waiting on results."
3BGxpglWsmqchaJ4ICmkfRGCtfwaxLip,W,"Job is completed, but waiting on results."
5mXalcpohn9Pqy24rcalRM7I6z3WGzjM,W,"Job is completed, but waiting on results."
6eOtrXEBgqhcuDS9I2iQ1S3Jmn1Ux9Y4,W,"Job is completed, but waiting on results."
B9y0ZRATf7o0Wm9kErqq0BInuLnv5lxV,Q,"Job is queued, and is waiting for resources."
BEZrMcOksxU0Unkor34BMdp3G5eiLc1Z,W,"Job is completed, but waiting on results."
EEF7ZlOZuBNBGCjnAdomLKN9WXdwQOlQ,W,"Job is completed, but waiting on results."
gCNLTryVFr9d6tvOARRKtjYT28t05RRa,W,"Job is completed, but waiting on results."


## Step 6: Get the results

Running the cell below will get the output files from our job

In [ ]:
import get_results

get_results.getResults(job_id[0], filename="output.tgz", blocking=True)

## Step 7: Output

In [ ]:
!tar zxf output.tgz

In [ ]:
!cat stdout.log